## Data Acquisition and Cleaning

This project relys on two different sources of data: United States' Bureau of Transport Statistics, and National Climatic Data Center. The fisrt source can only be accessed via its website, while the second provides an API with an extensive guide for utilising it.

Bureau of Transport Statistics provides extensive statistics on various forms of transportation, including airline performance data which consists of various figures related to departure and arrival delays. Data related to any year back to 1987 can be accessed. However, given that no API is provided, the relevant data should be downloaded via a menu system.

In [ ]:
impoting the necessary python libraries
import pandas as pd
import glob

In [ ]:
#creating a list containing all the csv files and sorting them alphabetically
csv_list = glob.glob('../../../../resource-datasets/capstone-data/flight-data/[0-9][0-9].csv')
csv_list.sort()

In [ ]:
#creating a dictionary containing the monthly weather share of delays labeled as National Aviation System delays
weather_share_nas = {'2017-01': 0.6617,
                     '2017-02': 0.6814,
                     '2017-03': 0.6475,
                     '2017-04': 0.5684,
                     '2017-05': 0.5320,
                     '2017-06': 0.6302,
                     '2017-07': 0.7192,
                     '2017-08': 0.7191,
                     '2017-09': 0.6112,
                     '2017-10': 0.5916,
                     '2017-11': 0.6567,
                     '2017-12': 0.5707,
                     '2018-01': 0.6056,
                     '2018-02': 0.6057,
                     '2018-03': 0.5891,
                     '2018-04': 0.6408,
                     '2018-05': 0.7559,
                     '2018-06': 0.6445,
                     '2018-07': 0.7273,
                     '2018-08': 0.7956,
                     '2018-09': 0.6346,
                     '2018-10': 0.6169,
                     '2018-11': 0.6599,
                     '2018-12': 0.6391,
                     '2019-01': 0.6986,
                     '2019-02': 0.6886,
                     '2019-03': 0.6188,
                     '2019-04': 0.7336,
                     '2019-05': 0.7774,
                     '2019-06': 0.7497,
                     '2019-07': 0.7903,
                     '2019-08': 0.7390,
                     '2019-09': 0.5391,
                     '2019-10': 0.5939,
                     '2019-11': 0.6203,
                     '2019-12': 0.6742}

In [ ]:
#loading the first csv file, changing the column names to lower case, renaming the weather_delay to
#extreme_weather_delay for greater accuracy and extracting the weather share of NAS delays
flight_set = pd.read_csv(csv_list[0])
flight_set.columns = flight_set.columns.str.lower()
flight_set.rename(columns={'weather_delay': 'extreme_weather_delay'}, inplace=True)
flight_set['weather_delay'] = flight_set['nas_delay'] * weather_share_nas[list(weather_share_nas.keys())[0]]

In [ ]:
#setting up a loop to load the remaining files
for i, file in enumerate(csv_list[1:]):
    #loading the remaining files and concatenating the contents one by one into a single dataframe
    next_flight_set = pd.read_csv(file)
    next_flight_set.columns = next_flight_set.columns.str.lower()
    next_flight_set.rename(columns={'weather_delay': 'extreme_weather_delay'}, inplace=True)
    next_flight_set['weather_delay'] = next_flight_set['nas_delay'] * weather_share_nas[list(weather_share_nas.keys())[i]]
    flight_set = pd.concat([flight_set, next_flight_set])

In [ ]:
#finding the top twenty airports of origin and forming a new dataframe containing the data related to these airports only
top_twenty_origins = list(flight_set['origin'].value_counts().sort_values(ascending=False).head(20).index)
top_twenty_airports = flight_set[flight_set['origin'].isin(top_twenty_origins)]

In [ ]:
#converting the schedulled departure time into a string
top_twenty_airports['crs_dep_time'] = top_twenty_airports['crs_dep_time'].astype(str)

In [ ]:
#defining a function to transform the schedulled departure time to a consistant 4-digit format
def correct_time(cell):
    if len(cell) == 3:
        return '0'+cell
    elif len(cell) == 2:
        return '00'+cell
    elif len(cell) == 1:
        return '000'+cell
    else:
        return cell

In [ ]:
#using the above function to correct the formatting
top_twenty_airports['crs_dep_time'] = top_twenty_airports['crs_dep_time'].apply(correct_time)
#converting the schedulled departure time to hh:mm:ss format
top_twenty_airports['crs_dep_time'] = top_twenty_airports['crs_dep_time'].apply(lambda x: x[0: 2]+':'+x[2:]+':00')
#combining the schedulled departure time with the departure date in order to create a single timestamp
top_twenty_airports['dep_timestamp'] = pd.to_datetime(top_twenty_airports['fl_date']+' '+top_twenty_airports['crs_dep_time'])

In [ ]:
#droping the that are no longer required and sorting the dataframe by origin
top_twenty_airports.drop(['unnamed: 25', 'fl_date', 'crs_dep_time', 'dep_time'], axis=1, inplace=True)
top_twenty_airports.sort_values('origin', inplace=True)

In [ ]:
#correcting the order inwhich the columns appear in the dataframe
top_twenty_airports = top_twenty_airports[['dep_timestamp', 'op_unique_carrier', 'tail_num', 'origin', 'origin_city_name',
                                            'dest', 'dest_city_name', 'dep_delay_new', 'wheels_off', 'wheels_on',
                                            'taxi_in', 'arr_delay_new', 'cancelled', 'cancellation_code', 'diverted',
                                            'air_time', 'distance', 'carrier_delay', 'extreme_weather_delay', 'weather_delay',
                                            'nas_delay', 'security_delay', 'late_aircraft_delay']]

In [ ]:
#replacing the missing values in the various delay columns to zero
top_twenty_airports['carrier_delay'].fillna(0, inplace=True)
top_twenty_airports['extreme_weather_delay'].fillna(0, inplace=True)
top_twenty_airports['weather_delay'].fillna(0, inplace=True)
top_twenty_airports['nas_delay'].fillna(0, inplace=True)
top_twenty_airports['security_delay'].fillna(0, inplace=True)
top_twenty_airports['late_aircraft_delay'].fillna(0, inplace=True)

#writing the dataframe into a csv file
top_twenty_airports.to_csv('../../../../resource-datasets/capstone-data/flight-data/top-twenty-airports.csv', index=False)